<a href="https://colab.research.google.com/github/vasjuta/SA/blob/master/SentimentAnalysisForProduction_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving data.csv to data.csv
User uploaded file "data.csv" with length 1056567 bytes


In [0]:
##Run only positive and only negative - trained data
import pandas as pd
import numpy as np


data_rows = pd.read_csv('data_for_all_states.csv', sep=',',error_bad_lines=False,usecols=['content', 'label'],encoding="ISO-8859-1")

print(data_rows[:3])

data = [[row[0].lower() , row[1]]
#            .replace(",", "").replace(".", "").replace("!", "").replace("?", "")
#            .replace(";", "").replace(":", "").replace("*", "")
#            .replace("(", "").replace(")", "")
#            .replace("/", ""), row[1]]
           for row in data_rows.values[:len(data_rows)]]

  
np.random.shuffle(data) 
# data_for_test = data[:5000]
# training = data[5000:]


raw_data =  [item[0] for item in data] 
labels =  [item[1] for item in data]

print(len(raw_data))
print(len(labels))
print(raw_data[:3])
print(labels[:3])


# train_data =  [item[0] for item in training] 
# train_label =  [item[1] for item in training] 
    
# test_data =  [item[0] for item in data_for_test] 
# label_data =  [item[1] for item in data_for_test] 


                                             content  label
0  This month, get your collagen mask complimenta...      2
1  It's a mood booster and nausea reliever - you ...      4
2  #overload but I've heard good things about the...      4
10126
10126
['a somewhat disappointing sheet mask for the night_íä\x89å\x8c_', '#champagneshowerstt #organicskincare #allnatural #madewithlove #madefresh #facemasksarelife #bodyscrubs #bodyoils #faceserums #lipscrubs.', 'great skin is not simply a matter of dna your daily habits, in fact have a big impact on what you see in the mirror.']
[0, 2, 4]


In [0]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

cvec=CountVectorizer(min_df=5, ngram_range = (1,2))


# text_clf = Pipeline([('vect', cvec),
#                      ('tfidf', TfidfTransformer()),
#                      ('clf', MultinomialNB())])  # BernoulliNB

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2), (2,3)],
#     'vect__ngram_range': [(1,2),(2, 2), (2, 3)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
#     'clf__alpha': [1, 1e-1, 1e-2]
}

In [0]:
x_train, x_test, y_train, y_test = train_test_split(raw_data, labels, test_size=0.33, random_state=42)

In [0]:
#prepare the test data
data_rows = pd.read_csv('a1389327_SplitterAPI_Splitting_all_bulks_Social_10k.csv', sep=',',error_bad_lines=False,usecols=['content', 'label'],encoding="ISO-8859-1")
test_data = data_rows['content'].values[:len(data_rows)]
label_data = data_rows['label'].values[:len(data_rows)]

In [0]:
# !pip install hadoop

In [0]:
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from mlxtend.preprocessing import DenseTransformer
from sklearn.base import TransformerMixin 
from scipy.sparse import csr_matrix
from sklearn.externals import joblib

# feature_names = cvec.get_feature_names()
      
      
score = 'f1_macro'
print("# Tuning hyper-parameters for %s" % score)
print()
np.errstate(divide='ignore')


text_clf = Pipeline([('vect', cvec),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LogisticRegression())])  # BernoulliNB


clf = GridSearchCV(text_clf, tuned_parameters, cv=10, scoring=score)
clf.fit(raw_data, labels)

# joblib.dump(clf, 'trained_model.pkl') 

# clf = joblib.load('trained_model.pkl') 

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
for mean, std, params in zip(clf.cv_results_['mean_test_score'], 
                             clf.cv_results_['std_test_score'], 
                             clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_pred = clf.predict(test_data)
y_prob = clf.predict_proba(test_data)
# print("y_prob:" + str(y_prob))
print(classification_report(label_data, y_pred, digits=4))
print("Accuracy:",metrics.accuracy_score(label_data, y_pred))
print()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Tuning hyper-parameters for f1_macro



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed

Best parameters set found on development set:

{'tfidf__norm': 'l2', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

Grid scores on development set:

0.428 (+/-0.036) for {'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
0.337 (+/-0.022) for {'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
0.324 (+/-0.030) for {'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__ngram_range': (2, 2)}
0.301 (+/-0.018) for {'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__ngram_range': (2, 3)}
0.445 (+/-0.032) for {'tfidf__norm': 'l1', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
0.409 (+/-0.025) for {'tfidf__norm': 'l1', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
0.346 (+/-0.036) for {'tfidf__norm': 'l1', 'tfidf__use_idf': False, 'vect__ngram_range': (2, 2)}
0.327 (+/-0.027) for {'tfidf__norm': 'l1', 'tfidf__use_idf': False, 'vect__ngram_range': (2, 3)}
0.612 (+/-0.022) for {'tfidf__norm': 'l2', 'tfidf__use_idf': True, 'vect